In [ ]:
import pandas as pd
import numpy as np
import string
import scipy
import sklearn
import spacy
import nltk
import re
import os

In [ ]:
from nltk import word_tokenize
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import cosine_similarity
#from sklearn.feature_extraction.text import TfidfVectorizer
from sentence_transformers import SentenceTransformer, util

In [ ]:
def read_csv_files_in_folder(folder_path):
    """
    Read all CSV files in a folder and concatenate them into a single DataFrame.

    Parameters:
    - folder_path (str): Path to the folder containing CSV files.

    Returns:
    - pd.DataFrame: Concatenated DataFrame.
    """
    # List all files in the folder
    all_files = os.listdir(folder_path)

    # List to store DataFrames
    dfs = []

    # Loop through CSV files and read them
    for file in all_files:
        if file.endswith('.csv'):
            file_path = os.path.join(folder_path, file)
            df = pd.read_csv(file_path)
            dfs.append(df)

    # Concatenate DataFrames into one
    concatenated_df = pd.concat(dfs, ignore_index=True)
    
    return concatenated_df

In [ ]:
def text_embed(text_col_name, dataframe, vector_col_name):
    model = SentenceTransformer("thenlper/gte-small")
    dataframe[text_col_name] = dataframe[text_col_name].astype(str)
    dataframe[vector_col_name] = dataframe[text_col_name].apply(lambda x: model.encode(x))
    return dataframe

In [ ]:
# Set random seed for reproducibility
np.random.seed(42)

# Function to find the matching label for each corpus
def match_corpus_to_label(corpus_embedding, label_embeddings, threshold=0.85):
    similarities = cosine_similarity([corpus_embedding], label_embeddings)
    max_similarity = np.max(similarities)
    
    if max_similarity >= threshold:
        # Get the index of the matching label
        label_index = np.argmax(similarities)
        return label_index
    else:
        return None

In [ ]:
# Set random seed for reproducibility
np.random.seed(42)

def assign_subtopic(corpus_df, corpus_embeddings, subtopic_df, subtopic_embeddings, subtopic_index_reference, subtopic_output):
    # Iterate over each row in sample_data
    for index, row in corpus_df.iterrows():
        corpus_embedding = row[corpus_embeddings]  # Assuming 'QA_vector' is the column with corpus embeddings
    
        # If the embedding is a sequence or array, stack them vertically and compute the mean
        if isinstance(corpus_embedding, list):
            corpus_embedding = np.mean(np.vstack(corpus_embedding), axis=0)

        # If the embedding is still 2D, flatten it to 1D
        if len(corpus_embedding.shape) > 1:
            corpus_embedding = corpus_embedding.flatten()

        matching_label_index = match_corpus_to_label(corpus_embedding, np.vstack(subtopic_embeddings.values))

        # If a matching label is found, assign it to a new column 'AssignedLabel'
        if matching_label_index is not None:
            corpus_df.at[index, subtopic_output] = subtopic_df.at[matching_label_index, subtopic_index_reference]

In [ ]:
def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()

    # Remove numbers, symbols, and punctuation (except for the case where 2 follows CO)
    text = re.sub(r'[\d' + re.escape(string.punctuation) + '](?<![cC][oO]2)', '', text)

    # Tokenize the text
    tokens = word_tokenize(text)

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]

    # Join the tokens back into a single string
    preprocessed_text = ' '.join(tokens)

    return preprocessed_text

# SMART CITIES

In [ ]:
smart_cities_df = pd.read_excel(r"C:\Users\RedHat\Downloads\Tiiqu\Supercleaned demo dataset ready for Topic mod 21 03.xlsx", 
                       sheet_name=1)
smart_cities_df

In [ ]:
smart_cities_df.columns

In [ ]:
smart_cities_df['QA'] = smart_cities_df['Question'] + ' ' + smart_cities_df['Answer']
smart_cities_df

In [ ]:
# smart cities subtopic
folder_path = r'C:\Users\RedHat\Downloads\Tiiqu\Subtopics'
smcts_subtopic_df = read_csv_files_in_folder(folder_path)
smcts_subtopic_df

In [ ]:
# text embedding for smart cities
text_embed('QA', smart_cities_df, 'QA_vector')

In [ ]:
# subtopic embedding for smart cities
text_embed('Description', smcts_subtopic_df, 'Des_vector')

In [ ]:
# Subtopic assignment for smart cities dataset
assign_subtopic(smart_cities_df, 'QA_vector', smcts_subtopic_df, smcts_subtopic_df['Des_vector'], 'Terminology', 'Assigned Subtopic')

In [ ]:
# PREPROCESSING

In [ ]:
smart_cities_df['QA_prep'] = smart_cities_df['QA'].apply(preprocess_text)

In [ ]:
smcts_subtopic_df['Des_prep'] = smcts_subtopic_df['Description'].apply(preprocess_text)

In [ ]:
# text embedding for preprocessed smart cities corpus
text_embed('QA_prep', smart_cities_df, 'QA_prep_vector')

In [ ]:
# subtopic embedding for preprocessed smart cities description
text_embed('Des_prep', smcts_subtopic_df, 'Des_prep_vector')

In [ ]:
#Subtopic assignment for smart cities dataset
assign_subtopic(smart_cities_df, 'QA_prep_vector', smcts_subtopic_df, smcts_subtopic_df['Des_prep_vector'], 'Terminology', 'Assigned Subtopic(Preprocessed)')

In [ ]:
smart_cities_df

In [ ]:
# Export to CSV
smart_cities_df.to_csv('Smart Cities SUBTOPIC ALLOCATION with New Dataset---85percent.csv')

In [ ]:
subs = pd.read_excel(r"C:\Users\RedHat\Downloads\Tiiqu\Smart cities macro topic, topics, sub-topics.xlsx", sheet_name=1)
subs

In [ ]:
UM = subs['Urban Mobility'].values
ITS = subs['Intelligent Transportation Systems'].values
SG = subs['Smart Grids'].values
EM = subs['Energy Management'].values
SI = subs['Sustainable Infrastructure'].values
DG = subs['Digital Governance'].values
IiC = subs['IoT in Cities'].values
SB = subs['Smart Buildings'].values
WM = subs['Waste Management'].values
WaM = subs['Water Management'].values
CE = subs['Civic Engagement'].values
PS = subs['Public Safety'].values
EvM = subs['Environmental Monitoring'].values
UPA = subs['Urban Planning Analytics'].values
SSL = subs['Smart Street Lighting'].values
UGS = subs['Urban Green Spaces'].values
EGS = subs['E-Government Services'].values
AQM = subs['Air Quality Monitoring'].values
SPS = subs['Smart Parking Solutions'].values
UAV = subs['Urban autonomous vehicles'].values

In [ ]:
def topic_allocation(sub_topic):
    if pd.isnull(sub_topic):
        return "NA"     
    if any(keyword in sub_topic for keyword in UM):
        return "Urban Mobility"
    elif any(keyword in sub_topic for keyword in ITS):
        return "Intelligent Transportation Systems"
    elif any(keyword in sub_topic for keyword in SG):
        return "Smart Grids"
    elif any(keyword in sub_topic for keyword in EM):
        return "Energy Management"
    elif any(keyword in sub_topic for keyword in SI):
        return "Sustainable Infrastructure"
    elif any(keyword in sub_topic for keyword in DG):
        return "Digital Governance"
    elif any(keyword in sub_topic for keyword in IiC):
        return "IoT in Cities"
    elif any(keyword in sub_topic for keyword in SB):
        return "Smart Buildings"
    elif any(keyword in sub_topic for keyword in WM):
        return "Waste Management"
    elif any(keyword in sub_topic for keyword in WaM):
        return "Water Management"
    elif any(keyword in sub_topic for keyword in CE):
        return "Civic Engagement"
    elif any(keyword in sub_topic for keyword in PS):
        return "Public Safety"
    elif any(keyword in sub_topic for keyword in EvM):
        return "Environmental Monitoring"
    elif any(keyword in sub_topic for keyword in UPA):
        return "Urban Planning Analytics"
    elif any(keyword in sub_topic for keyword in SSL):
        return "Smart Street Lighting"
    elif any(keyword in sub_topic for keyword in UGS):
        return "Urban Green Spaces"
    elif any(keyword in sub_topic for keyword in EGS):
        return "E-Government Services"
    elif any(keyword in sub_topic for keyword in AQM):
        return "Air Quality Monitoring"
    elif any(keyword in sub_topic for keyword in SPS):
        return "Smart Parking Solutions"
    elif any(keyword in sub_topic for keyword in UAV):
        return "Urban autonomous vehicles"
    else:
        return "Other"

In [ ]:
smart_cities_df['Topic'] = smart_cities_df['Assigned Subtopic(Preprocessed)'].apply(topic_allocation)

In [ ]:
smart_cities_df

In [ ]:
# Export unstripped dataset to CSV
smart_cities_df.to_csv('Smart Cities Unstripped Dataset---85percent.csv')

In [ ]:
smart_cities_df_striped = smart_cities_df[smart_cities_df['Topic'] != 'NA']
smart_cities_df_striped 

In [ ]:
smart_cities_df_striped = smart_cities_df_striped[['Question', 'Answer', 'Macrotopic ', 'Topic',
                                                   'Assigned Subtopic(Preprocessed)', 'Source Url']]
smart_cities_df_striped

In [ ]:
smart_cities_df_striped.rename(columns={'Assigned Subtopic(Preprocessed)': 'Subtopic'}, inplace=True)

In [ ]:
smart_cities_df_striped.to_csv('Smart Cities for SUPERCLEANED dataset 23-3-2024.csv')

In [ ]:
smart_cities_df.to_csv('Unstriped Smart Cities for SUPERCLEANED dataset 23-3-2024.csv')

# CLEAN ENERGY

In [ ]:
afc_energy_df = pd.read_excel(r"C:\Users\RedHat\Downloads\Tiiqu\Supercleaned demo dataset ready for Topic mod 21 03.xlsx", 
                       sheet_name=2)
afc_energy_df

In [ ]:
afc_energy_df['QA'] = afc_energy_df['Question'] + ' ' + afc_energy_df['Answer']
afc_energy_df

In [ ]:
sub_df1 = pd.read_excel(r"C:\Users\RedHat\Downloads\Tiiqu\clean energy topic and subtopics (2).xlsx", 
                                 sheet_name='subtopic description 1-15')
sub_df2 = pd.read_excel(r"C:\Users\RedHat\Downloads\Tiiqu\clean energy topic and subtopics (2).xlsx", 
                                 sheet_name='Subtopics description (15-30)')
afce_subtopic_df = pd.concat([sub_df1, sub_df2], axis=0).reset_index(drop=True)
afce_subtopic_df

In [ ]:
# text embedding for clean energy corpus
text_embed('QA', afc_energy_df, 'QA_vector')

In [ ]:
# subtopic embedding for clean energy
text_embed('Description', afce_subtopic_df, 'Des_vector')

In [ ]:
# Subtopic assignment for clean energy dataset
assign_subtopic(afc_energy_df, 'QA_vector', afce_subtopic_df, afce_subtopic_df['Des_vector'], 'Subtopic', 'Assigned Subtopic')

In [ ]:
afc_energy_df['QA_prep'] = afc_energy_df['QA'].apply(preprocess_text)

In [ ]:
afce_subtopic_df['Des_prep'] = afce_subtopic_df['Description'].apply(preprocess_text)

In [ ]:
# text embedding for preprocessed clean energy corpus
text_embed('QA_prep', afc_energy_df, 'QA_prep_vector')

In [ ]:
# subtopic embedding for preprocessed clean energy description
text_embed('Des_prep', afce_subtopic_df, 'Des_prep_vector')

In [ ]:
#Subtopic assignment for clean energy dataset
assign_subtopic(afc_energy_df, 'QA_prep_vector', afce_subtopic_df, afce_subtopic_df['Des_prep_vector'], 'Subtopic', 'Assigned Subtopic(Preprocessed)')

In [ ]:
subs_cleanenergy = pd.read_excel(r"C:\Users\RedHat\Downloads\Tiiqu\clean energy topic and subtopics (2).xlsx", 
                                 sheet_name=1)
subs_cleanenergy

In [ ]:
SPS = subs_cleanenergy['Solar Photovoltaic Systems']
SPS.dropna(inplace=True)

In [ ]:
subs_cleanenergy.columns

In [ ]:
SPS = SPS.values
WT = subs_cleanenergy['Wind Turbines'].values
HP = subs_cleanenergy['Hydropower Plants'].values
GE = subs_cleanenergy['Geothermal Energy'].values
BE = subs_cleanenergy['Biomass Energy'].values
TWE = subs_cleanenergy['Tidal and Wave Energy'].values
EST = subs_cleanenergy['Energy Storage Technologies'].values
SGT = subs_cleanenergy['Smart Grid Technologies'].values
EEL = subs_cleanenergy['Energy-Efficient Lighting'].values
EnST = subs_cleanenergy['Energy for sustainable Transportation'].values
EEV = subs_cleanenergy['Energy Electric Vehicles'].values
GB = subs_cleanenergy['Green Buildings'].values
EEA = subs_cleanenergy['Energy-efficient Appliances'].values
CCS = subs_cleanenergy['Carbon Capture and Storage'].values
GHP = subs_cleanenergy['Green Hydrogen Production'].values
SAE = subs_cleanenergy['Sustainable Agriculture Energy'].values
CEE = subs_cleanenergy['Circular Economy Energy'].values
EFC = subs_cleanenergy['Eco-Friendly Construction Materials'].values
OTE = subs_cleanenergy['Ocean Thermal Energy '].values
DER = subs_cleanenergy['Distributed Energy Resources'].values
MGD = subs_cleanenergy['Microgrids'].values
EMS = subs_cleanenergy['Energy Management Systems'].values
CEI = subs_cleanenergy['Clean Energy Integration'].values
CSP = subs_cleanenergy['Community Solar Projects'].values
UPCE = subs_cleanenergy['Urban planning clean energy '].values
CR = subs_cleanenergy['Climate Resilience'].values
WF = subs_cleanenergy[' Wind Farms'].values
CES = subs_cleanenergy['Clean Energy skills'].values
REC = subs_cleanenergy['Renewable Energy Certification'].values
COP = subs_cleanenergy['Carbon Offset Programs'].values

In [ ]:
missing_values = ['nan']
SPS_cleaned = SPS[~np.isin(SPS, missing_values)]

In [ ]:
SPS_cleaned

In [ ]:
def topic_allocation_clngy(sub_topic):
    if pd.isnull(sub_topic):
        return "NA"
    if any(keyword in sub_topic for keyword in SPS):
        return "Solar Photovoltaic Systems"
    elif any(keyword in sub_topic for keyword in WT):
        return "Wind Turbines"
    elif any(keyword in sub_topic for keyword in HP):
        return "Hydropower Plants"
    elif any(keyword in sub_topic for keyword in GE):
        return "Geothermal Energy"
    elif any(keyword in sub_topic for keyword in BE):
        return "Biomass Energy"
    elif any(keyword in sub_topic for keyword in TWE):
        return "Tidal and Wave Energy"
    elif any(keyword in sub_topic for keyword in EST):
        return "Energy Storage Technologies"
    elif any(keyword in sub_topic for keyword in SGT):
        return "Smart Grid Technologies"
    elif any(keyword in sub_topic for keyword in EEL):
        return "Energy-Efficient Lighting"
    elif any(keyword in sub_topic for keyword in EnST):
        return "Energy for sustainable Transportation"
    elif any(keyword in sub_topic for keyword in EEV):
        return "Energy Electric Vehicles"
    elif any(keyword in sub_topic for keyword in GB):
        return "Green Buildings"
    elif any(keyword in sub_topic for keyword in EEA):
        return "Energy-efficient Appliances"
    elif any(keyword in sub_topic for keyword in CCS):
        return "Carbon Capture and Storage"
    elif any(keyword in sub_topic for keyword in GHP):
        return "Green Hydrogen Production"
    elif any(keyword in sub_topic for keyword in SAE):
        return "Sustainable Agriculture Energy"
    elif any(keyword in sub_topic for keyword in CEE):
        return "Circular Economy Energy"
    elif any(keyword in sub_topic for keyword in EFC):
        return "Eco-Friendly Construction Materials"
    elif any(keyword in sub_topic for keyword in OTE):
        return "Ocean Thermal Energy"
    elif any(keyword in sub_topic for keyword in DER):
        return "Distributed Energy Resources"
    elif any(keyword in sub_topic for keyword in MGD):
        return "Microgrids"
    elif any(keyword in sub_topic for keyword in EMS):
        return "Energy Management Systems"
    elif any(keyword in sub_topic for keyword in CEI):
        return "Clean Energy Integration"
    elif any(keyword in sub_topic for keyword in CSP):
        return "Community Solar Projects"
    elif any(keyword in sub_topic for keyword in UPCE):
        return "Urban planning clean energy"
    elif any(keyword in sub_topic for keyword in CR):
        return "Climate Resilience"
    elif any(keyword in sub_topic for keyword in WF):
        return "Wind Farms"
    elif any(keyword in sub_topic for keyword in CES):
        return "Clean Energy skills"
    elif any(keyword in sub_topic for keyword in REC):
        return "Renewable Energy Certification"
    elif any(keyword in sub_topic for keyword in COP):
        return "Carbon Offset Programs"
    else:
        return "Other"


In [ ]:
afc_energy_df['Topic'] = afc_energy_df['Assigned Subtopic(Preprocessed)'].apply(topic_allocation_clngy)

In [ ]:
afc_energy_df

In [ ]:
afc_energy_df_striped = afc_energy_df[afc_energy_df['Topic'] != 'NA']

In [ ]:
afc_energy_df_striped = afc_energy_df_striped[['Question', 'Answer', 'Macrotopic ', 'Topic',
                                                   'Assigned Subtopic(Preprocessed)', 'Source Url']]

In [ ]:
afc_energy_df_striped.rename(columns={'Assigned Subtopic(Preprocessed)': 'Subtopic'}, inplace=True)
afc_energy_df_striped = afc_energy_df_striped.reset_index(drop=True)

In [ ]:
afc_energy_df_striped

In [ ]:
afc_energy_df_striped.to_csv('Clean Energy for SUPERCLEANED dataset 23-3-2024.csv')

In [ ]:
afc_energy_df.to_csv('Unstriped Clean Energy for SUPERCLEANED dataset 23-3-2024.csv')

# CLIMATE ACTION

In [ ]:
climate_action_df = pd.read_excel(r"C:\Users\RedHat\Downloads\Tiiqu\Supercleaned demo dataset ready for Topic mod 21 03.xlsx", 
                       sheet_name=3)
climate_action_df

In [ ]:
climate_action_df['QA'] = climate_action_df['Question'] + ' ' + climate_action_df['Answer']
climate_action_df

In [ ]:
# Climate change subtopic
clmtaction_subtopic = pd.read_excel(r"C:\Users\RedHat\Downloads\Tiiqu\Climate Change macro-topic, topic, sub-topics .xlsx", 
                                 sheet_name=2, skiprows=[1])
clmtaction_subtopic

In [ ]:
# text embedding for climate action corpus
text_embed('QA', climate_action_df, 'QA_vector')

In [ ]:
# subtopic embedding for climate change
text_embed('Description', clmtaction_subtopic, 'Des_vector')
clmtaction_subtopic

In [ ]:
# Subtopic assignment for climate change dataset
assign_subtopic(climate_action_df, 'QA_vector', clmtaction_subtopic, clmtaction_subtopic['Des_vector'], 'Subtopic', 'Assigned Subtopic')

In [ ]:
climate_action_df

In [ ]:
climate_action_df['QA_prep'] = climate_action_df['QA'].apply(preprocess_text)

In [ ]:
clmtaction_subtopic['Des_prep'] = clmtaction_subtopic['Description'].apply(preprocess_text)

In [ ]:
# text embedding for preprocessed climate change corpus
text_embed('QA_prep', climate_action_df, 'QA_prep_vector')

In [ ]:
# subtopic embedding for preprocessed climate change description
text_embed('Des_prep', clmtaction_subtopic, 'Des_prep_vector')

In [ ]:
#Subtopic assignment for smart cities dataset
assign_subtopic(climate_action_df, 'QA_prep_vector', clmtaction_subtopic, clmtaction_subtopic['Des_prep_vector'], 'Subtopic', 'Assigned Subtopic(Preprocessed)')

In [ ]:
climate_action_df

In [ ]:
subs_climatechg = pd.read_excel(r"C:\Users\RedHat\Downloads\Tiiqu\Climate Change macro-topic, topic, sub-topics .xlsx", 
                                 sheet_name=0)
subs_climatechg

In [ ]:
subs_climatechg.columns

In [ ]:
GW = subs_climatechg['Global Warming'].dropna().values
SLR = subs_climatechg['Sea Level Rise'].dropna().values
CFR = subs_climatechg['Carbon Footprint Reduction'].dropna().values
SAP = subs_climatechg['Sustainable Agriculture Practices'].dropna().values
DAR = subs_climatechg['Deforestation and Reforestation'].dropna().values
MCG = subs_climatechg['Melting Ice Caps and Glaciers'].dropna().values
EWE = subs_climatechg['Extreme Weather Events'].dropna().values
BC = subs_climatechg['Biodiversity Conservation'].dropna().values
OA = subs_climatechg['Ocean Acidification'].dropna().values
CRI = subs_climatechg['Climate Resilient Infrastructure'].dropna().values
GER = subs_climatechg['Greenhouse Gas Emission Reduction'].dropna().values
CAS = subs_climatechg['Climate Change Adaptation Strategies'].dropna().values
CET = subs_climatechg['Clean Energy Transition'].dropna().values
ST = subs_climatechg['Sustainable Transportation'].dropna().values
CEI = subs_climatechg['circular Economy Initiatives'].dropna().values
CJE = subs_climatechg['Climate Justice and Equity'].dropna().values
CCS = subs_climatechg['carbon capture and Storage'].dropna().values
ICA = subs_climatechg['International Climate agreement '].dropna().values

In [ ]:
GW

In [ ]:
def topic_allocation_clmtactn(sub_topic):
    if pd.isnull(sub_topic):
        return "NA"
    if any(keyword in sub_topic for keyword in GW):
        return "Global Warming"
    elif any(keyword in sub_topic for keyword in SLR):
        return "Sea Level Rise"
    elif any(keyword in sub_topic for keyword in CFR):
        return "Carbon Footprint Reduction"
    elif any(keyword in sub_topic for keyword in SAP):
        return "Sustainable Agriculture Practices"
    elif any(keyword in sub_topic for keyword in DAR):
        return "Deforestation and Reforestation"
    elif any(keyword in sub_topic for keyword in MCG):
        return "Melting Ice Caps and Glaciers"
    elif any(keyword in sub_topic for keyword in EWE):
        return "Extreme Weather Events"
    elif any(keyword in sub_topic for keyword in BC):
        return "Biodiversity Conservation"
    elif any(keyword in sub_topic for keyword in OA):
        return "Ocean Acidification"
    elif any(keyword in sub_topic for keyword in CRI):
        return "Climate Resilient Infrastructure"
    elif any(keyword in sub_topic for keyword in GER):
        return "Greenhouse Gas Emission Reduction'"
    elif any(keyword in sub_topic for keyword in CAS):
        return "Climate Change Adaptation Strategies"
    elif any(keyword in sub_topic for keyword in CET):
        return "Clean Energy Transition"
    elif any(keyword in sub_topic for keyword in ST):
        return "Sustainable Transportation"
    elif any(keyword in sub_topic for keyword in CEI):
        return "circular Economy Initiatives"
    elif any(keyword in sub_topic for keyword in CJE):
        return "Climate Justice and Equity"
    elif any(keyword in sub_topic for keyword in CCS):
        return "carbon capture and Storage"
    elif any(keyword in sub_topic for keyword in ICA):
        return "International Climate Agreements"
    else:
        return "Other"

In [ ]:
climate_action_df['Topic'] = climate_action_df['Assigned Subtopic(Preprocessed)'].apply(topic_allocation_clmtactn)

In [ ]:
climate_action_df_striped = climate_action_df[climate_action_df['Topic'] != 'NA']

In [ ]:
climate_action_df_striped = climate_action_df_striped[['Question', 'Answer', 'Macrotopic ', 'Topic',
                                                   'Assigned Subtopic(Preprocessed)', 'Source Url']]

In [ ]:
climate_action_df_striped.rename(columns={'Assigned Subtopic(Preprocessed)': 'Subtopic'}, inplace=True)
climate_action_df_striped = climate_action_df_striped.reset_index(drop=True)

In [ ]:
climate_action_df_striped

In [ ]:
climate_action_df_striped.to_csv('Climate Change for SUPERCLEANED dataset 23-3-2024.csv')

In [ ]:
climate_action_df.to_csv('Unstriped Climate Change for SUPERCLEANED dataset 23-3-2024.csv')